# Demonstrating PDF Journalling - Part 3
Journalling updates to a PDF is supported with MuPDF v1.19.*.

For a basic notebook session, see `journalling1.ipynb`.

In this notebook we demonstrate how to save a journalled PDF together with the journal such that both can be opened again later to continue journalled updating, undoing and redoing operations.

## Make journalled updates to an existing PDF and take a snapshot

In [40]:
import fitz
from pprint import pprint

if tuple(map(int, fitz.VersionBind.split("."))) < (1, 19, 0):
    raise ValueError("Need PyMuPDF v1.19.0 or higher")

doc = fitz.open("1page.pdf")  # work with an existing PDF
doc.journal_enable()  # enable journalling for it
doc.journal_start_op("new page")
page = doc.new_page()
doc.journal_stop_op()
# insert 5 text lines, each within its own operation:
for i in range(5):
    doc.journal_start_op("insert-%i" % i)
    page.insert_text((100, 100 + 20*i), "This is line %i." % i)
    doc.journal_stop_op()

Show journal status information: number of current operation, total operation count.

In [41]:
doc.journal_position()

(6, 6)

We now take a snapshot of the current PDF and its journal. Motivations may e.g. be submitting it to some review, before any redactions are applied, or similar purposes.

In [42]:
snapname = doc.name.replace(".pdf", "-snap1.pdf")
logname = doc.name.replace(".pdf", "-snap1.log")

doc.save_snapshot(snapname)
doc.journal_save(logname)
doc.close()

## Open snapshot PDF and its synchronized journal
The resulting file, `1page-snap1.pdf", is a valid PDF in every sense of the word: it can be displayed or printed, text can be extracted, etc.

The journal file is like a protocol of all changes applied in the last update session. Its validity is dependent on the the PDF snapshot **_not neing changed in any way._**

If this condition is met and snapshop PDF and its journal are still snychronized, both can be opened to continue journalled updates.

In [43]:
doc = fitz.open(snapname)  # open last update state of the PDF
doc.journal_load(logname)  # load the - hopefully compatible - journal

This has worked!

Now make a few checks to see what we have got:

In [44]:
print("Snapshot PDF '%s' contains the following update status:" % snapname)
print()
current, total = doc.journal_position()
for i in range(total):
    print("Operation %i: '%s'" % (i, doc.journal_op_name(i)))

actions = doc.journal_can_do()
print()
print("Possible actions:")
print("    undo: '%s'" % actions["undo"])
print("    redo: '%s'" % actions["redo"])

Snapshot PDF '1page-snap1.pdf' contains the following update status:

Operation 0: 'new page'
Operation 1: 'insert-0'
Operation 2: 'insert-1'
Operation 3: 'insert-2'
Operation 4: 'insert-3'
Operation 5: 'insert-4'

Possible actions:
    undo: 'True'
    redo: 'False'
